# EchoPro Bootstrapping Workflow

## Import libraries and configure the Jupyter notebook

In [1]:
# libraries used in the Notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 

# Python version of EchoPro
import EchoPro

# Allows us to grab the SemiVariogram class so we can use its models
from EchoPro.computation import SemiVariogram as SV

# Allows us to easily use matplotlib widgets in our Notebook
#%matplotlib widget

In [2]:
%%time
survey_2019 = EchoPro.Survey(init_file_path='../config_files/initialization_config.yml',
                             survey_year_file_path='../config_files/survey_year_2019_config.yml',
                             source=3, 
                             exclude_age1=True)

A full check of the initialization file contents needs to be done!
A check of the survey year file contents needs to be done!
CPU times: user 2.67 ms, sys: 3.35 ms, total: 6.02 ms
Wall time: 6.15 ms


In [3]:
%%time 
survey_2019.load_survey_data() 

CPU times: user 1.55 s, sys: 5.56 ms, total: 1.55 s
Wall time: 1.55 s


## Get bootstrapping object
- Obtain a bootstrapping object tied to a specific survey year

In [4]:
boot_obj = survey_2019.get_bootstrapping()

## Run bootstrapping without Kriging
In EchoPro we can perform bootstrapping on a reduced number of transects and obtain the areal biomass density for each iteration of bootstrapping as well as the associated mean Jolly-Hampton CV value. 

In [5]:
%%time
boot_results_no_krig = boot_obj.run_bootstrapping(run_kriging=False,
                                                  removal_percentage=50.0,
                                                  num_iterations=5,
                                                  seed=1234)

CPU times: user 9.74 s, sys: 33.4 ms, total: 9.77 s
Wall time: 9.77 s


In [6]:
boot_results_no_krig

,tot_biomass_no_kriging,CV_JH_mean_no_kriging
iteration,,
1,4.823803e+08,0.259509
2,6.287572e+08,0.299433
3,4.268084e+08,0.277257
4,6.319543e+08,0.260371
5,5.654719e+08,0.281114


In [7]:
print(f"Mean total biomass without Kriging: {1e-6 * boot_results_no_krig['tot_biomass_no_kriging'].mean():.3f} kmt")

Mean total biomass without Kriging: 547.074 kmt


## Run bootstrapping with Kriging
Now, we demonstrate how to run bootstrapping with Kriging 

### Initialize Kriging Mesh
Before we run bootstrapping we need to establish what Kriging mesh we will be using. Note: In bootstrapping the transformation of all meshes will be done automatically.   

In [8]:
krig_mesh = survey_2019.get_kriging_mesh()

### Set appropriate bootstrap Kriging parameters and run boostrapping 

In [9]:
bootstrap_kriging_params = dict(
    # kriging parameters
    k_max=10,
    k_min=3,
    R=0.0226287,
    ratio=0.001,
    
    # parameters for semi-variogram model
    s_v_params={'nugget': 0.0, 'sill': 0.95279, 'ls': 0.0075429,
                'exp_pow': 1.5, 'ls_hole_eff': 0.0},
    
    # grab appropriate semi-variogram model
    s_v_model=SV.generalized_exp_bessel,
    
    # set Kriging mesh
    krig_mesh=krig_mesh 
)

In [10]:
%%time
boot_results_krig = boot_obj.run_bootstrapping(run_kriging=True,
                                               kriging_params=bootstrap_kriging_params,
                                               removal_percentage=50.0,
                                               num_iterations=5,
                                               seed=1234)

CPU times: user 31.9 s, sys: 4.39 s, total: 36.3 s
Wall time: 28.4 s


In [11]:
boot_results_krig

,tot_biomass_no_kriging,CV_JH_mean_no_kriging,tot_biomass_kriging,CV_JH_mean_kriging
iteration,,,,
1,4.823803e+08,0.259458,1.060860e+09,0.135964
2,6.287572e+08,0.299280,1.687877e+09,0.150531
3,4.268084e+08,0.277408,1.056840e+09,0.140598
4,6.319543e+08,0.260353,1.594250e+09,0.135785
5,5.654719e+08,0.281390,1.470527e+09,0.155058


In [12]:
print(f"Mean total biomass with Kriging: {1e-6 * boot_results_krig['tot_biomass_kriging'].mean():.3f} kmt")

Mean total biomass with Kriging: 1374.071 kmt
